In [14]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [16]:
embedding_function = OpenAIEmbeddings()
db_connection = Chroma(persist_directory="./mk-ultra/", embedding_function=embedding_function)

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [18]:
# LLM to use for compression
llm = ChatOpenAI(temperature=0)

# LLM -> LLMchain extractor
compressor = LLMChainExtractor.from_llm(llm=llm)
# Contextual Compression
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db_connection.as_retriever())

In [19]:
docs = db_connection.similarity_search('when was this declassified?') # normal way
# docs[0]

In [20]:
compressed_docs = compression_retriever.get_relevant_documents("When was this declassified?")

/Users/pete/.local/share/virtualenvs/langchain-udemy-notebooks-Raz3oF7P/lib/python3.11/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [21]:
compressed_docs[0]

Document(page_content='Transcriptions of Declassified Project Artichoke Documents at the Wayback Machine (archived May 1, 2011)', metadata={'source': 'https://en.wikipedia.org/wiki/Project_Artichoke', 'summary': "Project Artichoke (also referred to as Operation Artichoke) was a project developed and enacted by the United States Central Intelligence Agency (CIA) for the purpose of researching methods of interrogation.\nInitially known as Project Bluebird, Project Artichoke officially arose on August 20, 1951, and was operated by the CIA's Office of Scientific Intelligence. The primary goal of Project Artichoke was to determine whether a person could be involuntarily made to perform an act of attempted assassination. The project also studied the effects of hypnosis, forced addiction to (and subsequent withdrawal from) morphine, and other chemicals, including LSD, to produce amnesia and other vulnerable states in subjects.\nProject Artichoke was succeeded by Project MKUltra, which began i